In [2]:
import numpy as np
import pandas  as pd


from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical


In [3]:

data = pd.read_csv("AB_NYC_2019.csv")

sample_data = data.sample(frac=0.1, random_state=42)

remaining_data = data.drop(sample_data.index)

sample_data_no_price = sample_data.drop(columns=['price'])
sample_data_no_price.to_csv("sample_no_price.csv", index=False)

remaining_data.to_csv("test_data.csv",index=False)

In [4]:
data.head(10)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
5,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129
6,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,60,45,49,2017-10-05,0.40,1,0
7,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,2,430,2019-06-24,3.47,1,220
8,5203,Cozy Clean Guest Room - Family Apt,7490,MaryEllen,Manhattan,Upper West Side,40.80178,-73.96723,Private room,79,2,118,2017-07-21,0.99,1,0
9,5238,Cute & Cozy Lower East Side 1 bdrm,7549,Ben,Manhattan,Chinatown,40.71344,-73.99037,Entire home/apt,150,1,160,2019-06-09,1.33,4,188


In [5]:
data=data.drop(['id', 'name', 'host_id', 'host_name', 'last_review', 'latitude', 'longitude'],axis=1)



In [6]:


# 1. neighbourhood_group'e (şehir) 10'ar artan kod ver
city_encoder = LabelEncoder()
data['city_code'] = city_encoder.fit_transform(data['neighbourhood_group']) * 10

# 2. location_code sütununu baştan oluştur
data['location_code'] = -1  # başlangıçta geçici -1 değeri

# 3. Her şehir grubu için mahalleleri encode et
for group_code in data['city_code'].unique():
    mask = data['city_code'] == group_code
    hood_encoder = LabelEncoder()
    hood_codes = hood_encoder.fit_transform(data.loc[mask, 'neighbourhood'])
    
    # Şehir + mahalle kodunu topla → location_code
    data.loc[mask, 'location_code'] = group_code + hood_codes

# 4. Artık data DataFrame'i location_code sütununu içeriyor
print(data[['neighbourhood_group', 'neighbourhood', 'location_code']].head())


  neighbourhood_group neighbourhood  location_code
0            Brooklyn    Kensington             41
1           Manhattan       Midtown             37
2           Manhattan        Harlem             30
3            Brooklyn  Clinton Hill             23
4           Manhattan   East Harlem             24


In [7]:
print(data.columns)

Index(['neighbourhood_group', 'neighbourhood', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365', 'city_code',
       'location_code'],
      dtype='object')


In [8]:
print(data['location_code'])

0        41
1        37
2        30
3        23
4        24
         ..
48890    12
48891    20
48892    30
48893    31
48894    31
Name: location_code, Length: 48895, dtype: int64


In [9]:
data=data.drop(['neighbourhood_group', 'neighbourhood','city_code'],axis=1)




In [10]:
room_encoder = LabelEncoder()
data['room_type'] = room_encoder.fit_transform(data['room_type'])


In [11]:
data.head(5)

,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,location_code
0,1,149,1,9,0.21,6,365,41
1,0,225,1,45,0.38,2,355,37
2,1,150,3,0,NaN,1,365,30
3,0,89,1,270,4.64,1,194,23
4,0,80,10,9,0.10,1,0,24


In [12]:
print(data.columns)

Index(['room_type', 'price', 'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'location_code'],
      dtype='object')


In [13]:
features = ['room_type', 'price', 'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'location_code']

X = data[features]
y = data['price']

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# --- 1. Girdi Özelliklerini Belirle ---
features = ['room_type', 'minimum_nights', 'number_of_reviews',
            'reviews_per_month', 'calculated_host_listings_count',
            'availability_365', 'location_code']

X = data[features].astype(float)
y = data['price'].astype(float)

# --- 2. Hedef Veriyi Log Dönüştür ---
y_log = np.log1p(y)  # log(1 + y)

# --- 3. Girdileri Normalize Et ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# --- 4. Eğitim/Test Böl ---
X_train, X_test, y_train_log, y_test_log = train_test_split(X_scaled, y_log, test_size=0.2, random_state=42)

# --- 5. Orijinal Test Verisini Kaydet (price olmadan) ---
original_test_data = data.iloc[y_test_log.index].copy()
original_test_data.drop(columns='price', inplace=True)
original_test_data.to_csv("test_data.csv", index=False)
print("✅ test_data.csv dosyası oluşturuldu (price sütunu olmadan).")

# --- 6. Modeli Kur ---
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  # Regresyon = aktivasyon yok
])

model.compile(loss='mse', optimizer='adam', metrics=['mae'])

# --- 7. Eğitimi Başlat ---
model.fit(X_train, y_train_log, epochs=100, batch_size=32, validation_split=0.1, verbose=1)

# --- 8. Tahmin Yap (loglu tahmin) ---
predictions_log = model.predict(X_test)
predictions = np.expm1(predictions_log)  # log(1 + y) -> y dönüşümü

# --- 9. Gerçek Fiyatlar (orijinal skala) ---
y_test_real = np.expm1(y_test_log)

# --- 10. Sonuçları Göster ---
results = pd.DataFrame({
    'Actual Price': y_test_real.values,
    'Predicted Price': predictions.flatten()
})

print("\n🎯 İlk 10 Tahmin:")
print(results.head(10))

# --- 11. Hata Ölçümleri ---
mae = mean_absolute_error(y_test_real, predictions)
rmse = np.sqrt(mean_squared_error(y_test_real, predictions))
print(f"\n📊 Ortalama Mutlak Hata (MAE): {mae:.2f}")
print(f"📉 Karekök Ortalama Kare Hata (RMSE): {rmse:.2f}")


✅ test_data.csv dosyası oluşturuldu (price sütunu olmadan).
Epoch 1/100
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 2s 934us/step - loss: 4.3315 - mae: 1.3552 - val_loss: 0.4801 - val_mae: 0.5379
Epoch 2/100
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 1s 848us/step - loss: 0.4924 - mae: 0.5545 - val_loss: 0.4774 - val_mae: 0.5384
Epoch 3/100
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 1s 830us/step - loss: 0.4910 - mae: 0.5529 - val_loss: 0.4775 - val_mae: 0.5399
Epoch 4/100
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 1s 835us/step - loss: 0.4921 - mae: 0.5535 - val_loss: 0.5101 - val_mae: 0.5485
Epoch 5/100
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 1s 833us/step - loss: 0.4969 - mae: 0.5554 - val_loss: 0.4779 - val_mae: 0.5381
Epoch 6/100
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 1s 827us/step - loss: 0.4966 - mae: 0.5551 - val_loss: 0.4786 - val_mae: 0.5378
Epoch 7/100
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 1s 855us/step - loss: 0.4996 - mae: 0.5565 - val_loss: 0.4827 - val_mae: 0.5384
Epoch 8/100
1101/1101 ━━━━━━━━━━━━━━━━━━━━ 1s 863us/step - loss: 0.4961 - mae: 